<a href="https://colab.research.google.com/github/alorber/Quadratic-Activation-Functions/blob/main/Quadratic_Activation_Functions_FINAL_CODE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Quadratic Neurons as Activation Functions in Deep Neural Networks**
## Andrew Lorber & Mark Koszykowski
### Based on *Pattern Classification using Quadratic Neuron: An Experimental Study* by Ganesh, Singh, and Murthy.


In [1]:
# Imports
import tensorflow as tf
import tensorflow.math as tfm
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Layer, InputLayer, Flatten, Dense, Conv2D, MaxPool2D, BatchNormalization
import numpy as np
import matplotlib.pyplot as plt

## Quadratic Neuron Layer Classes
Implementations of the Quadratic Activation Function

### Quadratic Neuron Layer
### Layer-Wide Coefficients

In [2]:
class Quadratic(Layer):
    def __init__(self, trainable=True, **kwargs):
        super(Quadratic, self).__init__(**kwargs)
        self.trainable = trainable
        # Initializes Coefficients
        self.coeffs = self.add_weight(shape=(1,6), initializer='random_normal', trainable=self.trainable)

    def call(self, inputs, mask=None):
        # Builds input
        x, y = tf.split(inputs, num_or_size_splits=2, axis=1)
        x2 = tfm.square(x)
        y2 = tfm.square(y)
        xy = tfm.multiply(x, y)

        quad_inputs = tf.stack([x2, xy, y2, x, y, tf.ones((tf.shape(x)))], axis=1)
        quad_outputs = tf.squeeze(tf.matmul(self.coeffs, quad_inputs), axis=[1])

        return quad_outputs

    def compute_output_shape(self, input_shape):
        return input_shape/2 

### Convolutional Quadratic Neuron Layer
### Layer-Wide Coefficients
Quadratic Activation Function for Convolutional Layer

In [3]:
class ConvQuadratic(Layer):
    def __init__(self, trainable=True, **kwargs):
        super(ConvQuadratic, self).__init__(**kwargs)
        self.trainable = trainable
        # Initializes Coefficients
        self.coeffs = self.add_weight(shape=(1,6), initializer='random_normal', trainable=self.trainable)

    def call(self, inputs, mask=None):
        # Saves dimensions to make code nicer
        batch_size = inputs.shape[0]
        height = inputs.shape[1]
        width = inputs.shape[2]
        num_filters = inputs.shape[3]

        # Reshapes so last 2 dimensions are a single filter
        inputs = tf.transpose(inputs, [0, 3, 1, 2])

        # Reshapes into columns for x,y
        inputs = tf.reshape(inputs, [batch_size, num_filters, -1, 2])

        # Transposes to get correct dimensions for matmul
        inputs = tf.transpose(inputs, [0,1,3,2])

        # Splits tensor into x & y
        x, y = tf.split(inputs,2,2)

        # Calculates other components of quadratic input
        x2 = tfm.square(x)
        y2 = tfm.square(y)
        xy = tfm.multiply(x, y)

        # Builds quadratic input
        quad_input = tf.concat([x2, xy, y2, x, y, tf.ones((tf.shape(x)))], axis=2)

        # Matmul -> Quadratic output
        quad_output = tf.matmul(self.coeffs, quad_input)

        # Reshapes back to original size with width / 2
        quad_output = tf.reshape(quad_output, [batch_size, num_filters, height, int(width/2)])

        # Rearranges axis to have num_filters last (as CONV2D expects)
        quad_output = tf.transpose(quad_output, [0,2,3,1])

        return quad_output

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[1], input_shape[2]/2, input_shape[3])

### Quadratic Neuron Layer
### Per-Neuron Coefficients
Each neuron has its own coefficients (as opposed to layer-wide coefficients).

In [4]:
class QuadraticPerN(Layer):
    def __init__(self, trainable=True, **kwargs):
        super(QuadraticPerN, self).__init__(**kwargs)
        self.trainable = trainable

    def build(self, input_shape):
        # Initializes Coefficients
        self.coeffs = self.add_weight(shape=(int(input_shape[1]/2),6), initializer='random_normal', trainable=self.trainable)

    def call(self, inputs, mask=None):
        # Builds input
        x, y = tf.split(inputs, num_or_size_splits=2, axis=1)
        x2 = tfm.square(x)
        y2 = tfm.square(y)
        xy = tfm.multiply(x, y)

        quad_inputs = tf.stack([x2, xy, y2, x, y, tf.ones((tf.shape(x)))], axis=1)
        quad_outputs = tf.reduce_sum(tf.multiply(self.coeffs, tf.transpose(quad_inputs, perm=(0,2,1))),2)

        return quad_outputs

    def compute_output_shape(self, input_shape):
        return input_shape/2 

### Convolutional Quadratic Neuron Layer
### Per-Neuron Coefficients
Quadratic Activation Function for Convolutional Layer

Each neuron has its own coefficients (as opposed to layer-wide coefficients).

In [5]:
class ConvQuadraticPerN(Layer):
    def __init__(self, trainable=True, **kwargs):
        super(ConvQuadraticPerN, self).__init__(**kwargs)
        self.trainable = trainable

    def build(self, input_shape):
        self.coeffs = self.add_weight(shape=(int(input_shape[2]/2) * input_shape[1],6), initializer='random_normal', trainable=self.trainable)

    def call(self, inputs, mask=None):
        # Saves dimensions to make code nicer
        batch_size = inputs.shape[0]
        height = inputs.shape[1]
        width = inputs.shape[2]
        num_filters = inputs.shape[3]

        # Reshapes so last 2 dimensions are a single filter
        inputs = tf.transpose(inputs, [0, 3, 1, 2])

        # Reshapes into columns for x,y
        inputs = tf.reshape(inputs, [batch_size, num_filters, -1, 2])

        # Splits tensor into x & y
        x, y = tf.split(inputs,2,3)

        # Calculates other components of quadratic input
        x2 = tfm.square(x)
        y2 = tfm.square(y)
        xy = tfm.multiply(x, y)

        # Builds quadratic input
        quad_input = tf.concat([x2, xy, y2, x, y, tf.ones((tf.shape(x)))], axis=3)

        # Gets inner product of each neuron and its coefficients
        quad_output = tf.reduce_sum(tf.multiply(self.coeffs, quad_input), 3)

        # Reshapes back to original size with width / 2
        quad_output = tf.reshape(quad_output, [batch_size, num_filters, height, int(width/2)])

        # Rearranges axis to have num_filters last (as CONV2D expects)
        quad_output = tf.transpose(quad_output, [0,2,3,1])

        return quad_output

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[1], input_shape[2]/2, input_shape[3])

## Model Builder Function
Builds models with given layer sizes, which are all exactly the same except for the layer activation functions. 

The activation functions included are: *Sigmoid*, *ReLu*, *Quadratic* (Layer-Wide Coefficients), *Quadratic* 
(Per-Neuron Coefficients)



In [6]:
# @params: layer_sizes = len is number of hidden layers, value is number of neurons at layer
#          layer_types = len is number of hidden layers, value is type of layer
#                       Options: Dense, Conv, MaxPool, Flatten
#          output_size = number of neurons at output layer
def buildModels(layer_sizes, layer_types, output_size = 10):

    # Sigmoid Model
    # --------------

    # Input Layer
    sigmoid_model = Sequential()

    if layer_types[0] == "Dense":
        sigmoid_model.add(Flatten())

    # Hidden Layers
    for (l,t) in zip(layer_sizes,layer_types):
        if t == "Dense":
            sigmoid_model.add(Dense(l, kernel_regularizer='l2', activation=tf.nn.sigmoid))
            sigmoid_model.add(BatchNormalization())
        elif t == "Conv": 
            sigmoid_model.add(Conv2D(l, 3, padding='same', kernel_regularizer='l2', activation=tf.nn.sigmoid))
            sigmoid_model.add(BatchNormalization())
        elif t == "MaxPool":
            sigmoid_model.add(MaxPool2D(l))
        elif t == "Flatten":
            sigmoid_model.add(Flatten())
    # Output Layer
    sigmoid_model.add(Dense(output_size, kernel_regularizer='l2', activation=tf.nn.softmax))


    # ReLu Model
    # -----------

    # Input Layer
    relu_model = Sequential()

    if layer_types[0] == "Dense":
        relu_model.add(Flatten())
    
    # Hidden Layers
    for (l,t) in zip(layer_sizes,layer_types):
        if t == "Dense":
            relu_model.add(Dense(l, kernel_regularizer='l2', activation=tf.nn.relu))
            relu_model.add(BatchNormalization())
        elif t == "Conv": 
            relu_model.add(Conv2D(l, 3, padding='same', kernel_regularizer='l2', activation=tf.nn.relu))
            relu_model.add(BatchNormalization())
        elif t == "MaxPool":
            relu_model.add(MaxPool2D(l))
        elif t == "Flatten":
            relu_model.add(Flatten())

    # Output Layer
    relu_model.add(Dense(output_size, kernel_regularizer='l2', activation=tf.nn.softmax))


    # Quadratic Model (Layer-wide Coefficients)
    # ---------------------------------------------

    # Input Layer
    layer_model = Sequential()

    if layer_types[0] == "Dense":
        layer_model.add(Flatten())
    
    # Hidden Layers
    for (l,t) in zip(layer_sizes,layer_types):
        if t == "Dense":
            layer_model.add(Dense(l, kernel_regularizer='l2', activation=Quadratic()))
            layer_model.add(BatchNormalization())
        elif t == "Conv": 
            layer_model.add(Conv2D(l, 3, padding='same', kernel_regularizer='l2', activation=ConvQuadratic()))
            layer_model.add(BatchNormalization())
        elif t == "MaxPool":
            layer_model.add(MaxPool2D(l))
        elif t == "Flatten":
            layer_model.add(Flatten())
    # Output Layer
    layer_model.add(Dense(output_size, kernel_regularizer='l2', activation=tf.nn.softmax))


    # Quadratic Model (Per-neuron Coefficients)
    # ---------------------------------------------

    # Input Layer
    neuron_model = Sequential()

    if layer_types[0] == "Dense":
        neuron_model.add(Flatten())
   
    # Hidden Layers
    for (l,t) in zip(layer_sizes,layer_types):
        if t == "Dense":
            neuron_model.add(Dense(l, kernel_regularizer='l2', activation=QuadraticPerN()))
            neuron_model.add(BatchNormalization())
        elif t == "Conv": 
            neuron_model.add(Conv2D(l, 3, padding='same', kernel_regularizer='l2', activation=ConvQuadraticPerN()))
            neuron_model.add(BatchNormalization())
        elif t == "MaxPool":
            neuron_model.add(MaxPool2D(l))
        elif t == "Flatten":
            neuron_model.add(Flatten())
    # Output Layer
    neuron_model.add(Dense(output_size, kernel_regularizer='l2', activation=tf.nn.softmax))


    # Prints Model Architecture
    for (l,t) in zip(layer_sizes, layer_types):
        print(f"{t} Layer - {l} Neurons")
        
    # Returns models
    return sigmoid_model, relu_model, layer_model, neuron_model

